# Train SVC on CPU, predict on GPU

This notebook trains SVC classifier with RFF on CPU, and predict on GPU.

In [1]:
import numpy as np
import sklearn.datasets
import sklearn.svm

# Import rfflearn.
import rfflearn.cpu
import rfflearn.gpu

## Prepare MNIST dataset

### Load MNIST dataset

Load the MNIST dataset using `sklearn.fetch_openml` and standardize it.
Also the loaded label data is expressed as a string, so it is converted to an integer type.

In [2]:
%%time

# Load MNIST.
Xs, ys = sklearn.datasets.fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False, data_home="./scikit_learn_data")

# Split to training and test data.
Xs_train, Xs_test, ys_train, ys_test = sklearn.model_selection.train_test_split(Xs, ys, test_size=10000, shuffle=False)

# Standardize the data (convert the input data range from [0, 255] to [0, 1]).
Xs_train = Xs_train.astype(np.float64) / 255.0
Xs_test  = Xs_test.astype(np.float64)  / 255.0

# Convert the string label to integer label.
ys_train = ys_train.astype(np.int32)
ys_test  = ys_test.astype(np.int32)

CPU times: total: 5.69 s
Wall time: 5.68 s


### Data dimension reduction

Reduce data dimension using PCA.
This step is not necessary, but contribute to the test accuracy.

In [3]:
%%time

dim_pca = 128

# Create matrix for principal component analysis.
_, V = np.linalg.eig(Xs_train.T @ Xs_train)
T = np.real(V[:, :dim_pca])

CPU times: total: 2.84 s
Wall time: 570 ms


## Train SVM model on CPU

Before transferring a RFFSVC model from CPU to GPU, train the RFFSVC model on CPU.

In [4]:
%%time

# SVM classifier with random Fourier features.
svc_cpu = rfflearn.cpu.RFFSVC(dim_kernel=1024, std_kernel=0.05)

# Train SVM.
svc_cpu.fit(Xs_train @ T, ys_train)

CPU times: total: 11 s
Wall time: 57.5 s


## Transfer SVM model from CPU to GPU

Convert the CPU model to GPU model.

In [5]:
svc_gpu = rfflearn.gpu.RFFSVC(svc_cpu=svc_cpu)

### Compare inference speed on CPU and GPU

At first, prepare the input data (after PCA conversion).

In [6]:
Xs = Xs_test @ T

### Inference on CPU

In [7]:
%%time

# Calculate score for test data.
_ = svc_cpu.predict(Xs)

CPU times: total: 1 s
Wall time: 395 ms


### Inference on GPU

In [8]:
%%time

# Calculate score for test data.
_ = svc_gpu.predict(Xs)

CPU times: total: 125 ms
Wall time: 41 ms
